# *k*-NN Editing  
An evaluation of the impact of two basic instance selection (case-base editing) algorithms.
- **Condensed Nearest Neighbour**  
Hart P. The condensed nearest neighbor rule (Corresp.). IEEE transactions on information theory. 1968 May;14(3):515-6.  
http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.294.6968&rep=rep1&type=pdf
- **Conservative Redundancy Removal**    
Delany, S. J., & Cunningham, P. (2004, August). An analysis of case-base editing in a spam filtering system. In European Conference on Case-Based Reasoning (pp. 128-141). Springer, Berlin, Heidelberg.
https://arrow.tudublin.ie/cgi/viewcontent.cgi?article=1034&context=scschcomcon


Basic implementations of both algorithms are available in `kNNEdit.py`.   
These implementations only work for binary classification tasks. 

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import kNNDataLoader
from kNNEdit import kNNCRR, kNNCNN
from collections import Counter

## Demonstration on 2D data  
AthleteSelection dataset

In [ ]:
athlete = pd.read_csv('AthleteSelection.csv',index_col = 'Athlete')
athlete = athlete.sample(frac=1) # Shuffle the data
#athlete

In [ ]:
y = athlete.pop('Selected').values
X = athlete.values
names = athlete.index
q = [5.0,7.5]
X.shape

In [ ]:
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
q_scaled = scaler.transform([q])
X_scaled.shape

### Conservative Redundancy Removal (CRR)
Using `kNNCRR` class from `kNNEdit.py`

In [ ]:
crr = kNNCRR()
X_crr, y_crr = crr.fit(X_scaled,y)
X.shape, X_crr.shape

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
color= ['red' if l == 'No' else 'green' for l in y]
x1 = X_scaled[:,0]
x2 = X_scaled[:,1]
xe1 = X_crr[:,0]
xe2 = X_crr[:,1]
plt.figure(figsize=(7,6))
plt.scatter(xe1,xe2, color='yellow', s = 100)
plt.scatter(x1,x2, color=color)
plt.scatter(q_scaled[0,0],q_scaled[0,1],color='black')
plt.annotate('q',(q_scaled[0,0]+0.05,q_scaled[0,1]))
plt.title("Athlete Selection (Normalized)")
plt.xlabel("Speed N(0,1)")
plt.ylabel("Agility N(0,1)")
plt.grid()
red_patch = mpatches.Patch(color='red', label='Not Selected')
blue_patch = mpatches.Patch(color='green', label='Selected')
plt.legend(handles=[red_patch, blue_patch],loc=4)
for i, txt in enumerate(names):
    plt.annotate(txt, (x1[i]+0.05, x2[i]))

Examples picked by Conservative Redundancy Removal marked in yellow. 

### Condensed Nearest Neighbour (CNN)
Using `kNNCNN` class from `kNNEdit.py`

In [ ]:
cnn = kNNCNN()
X_cnn, y_cnn = cnn.fit(X_scaled,y)
X.shape, X_cnn.shape

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
color= ['red' if l == 'No' else 'green' for l in y]
x1 = X_scaled[:,0]
x2 = X_scaled[:,1]
xe1 = X_crr[:,0]
xe2 = X_crr[:,1]
plt.figure(figsize=(7,6))
plt.scatter(xe1,xe2, color='yellow', s = 100)
plt.scatter(x1,x2, color=color)
plt.scatter(q_scaled[0,0],q_scaled[0,1],color='black')
plt.annotate('q',(q_scaled[0,0]+0.05,q_scaled[0,1]))
plt.title("Athlete Selection (Normalized)")
plt.xlabel("Speed N(0,1)")
plt.ylabel("Agility N(0,1)")
plt.grid()
red_patch = mpatches.Patch(color='red', label='Not Selected')
blue_patch = mpatches.Patch(color='green', label='Selected')
plt.legend(handles=[red_patch, blue_patch],loc=4)
for i, txt in enumerate(names):
    plt.annotate(txt, (x1[i]+0.05, x2[i]))

Examples picked by Condensed Nearest Neighbour shown in yellow. 

## Evaluation on three datasets

In [ ]:
kNN = KNeighborsClassifier()
crr = kNNCRR()
Name_arr, X_dir, y_dir = kNNDataLoader.data_loader()
Name_arr

In [ ]:
from sklearn.datasets import load_breast_cancer
bc = load_breast_cancer()
Name_arr = ['Breast','HTRU','Credit'] # Binary classification tasks only.
y_dir['Breast']=bc.target
X_dir['Breast']=bc.data

In [ ]:
Edit_sizes = {}
Edit_accs = {}
for ds in Name_arr:
    sizes = {}
    accs = {}
    X_train, X_test, y_train, y_test = train_test_split(X_dir[ds],y_dir[ds], test_size = 1/2)
    print(X_train.shape,X_test.shape)
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Full training data
    kNN = KNeighborsClassifier()
    kNN = kNN.fit(X_train,y_train)
    y_pred = kNN.predict(X_test)
    acc = accuracy_score(y_test,y_pred)
    print(ds,'Acc',acc)
    sizes['Full']= y_train.shape[0]
    accs['Full']= acc
    
    # CNN
    cnn = kNNCNN()
    X_cnn, y_cnn = cnn.fit(X_train,y_train)
    kNNcnn = kNN.fit(X_cnn,y_cnn)
    y_pred = kNNcnn.predict(X_test)
    print(y_test[:10],y_pred[:10])
    acc = accuracy_score(y_test,y_pred)
    print(ds,y_cnn.shape,'CNN Acc',acc)
    sizes['CNN']= y_cnn.shape[0]
    accs['CNN']= acc

    # CRR
    crr = kNNCRR()
    X_crr, y_crr = crr.fit(X_train,y_train)
    kNNcrr = kNN.fit(X_crr,y_crr)
    y_pred = kNNcrr.predict(X_test)
    print(y_test[:10],y_pred[:10])
    acc = accuracy_score(y_test,y_pred)
    print(ds,y_crr.shape,'Crr Acc',acc)
    sizes['CRR']= y_crr.shape[0]
    accs['CRR']= acc

    Edit_sizes[ds]=sizes
    Edit_accs[ds]=accs

In [ ]:
resA_pd = pd.DataFrame(Edit_accs)
resS_pd = pd.DataFrame(Edit_sizes)

In [ ]:
resA = resA_pd.T
resA['CNN%']=resA['CNN']/resA['Full']
resA['CRR%']=resA['CRR']/resA['Full']
resA

In [ ]:
resS = resS_pd.T
resS['CNN%']=resS['CNN']/resS['Full']
resS['CRR%']=resS['CRR']/resS['Full']
resS

In [ ]:
resA_pd

In [ ]:
ds = 'Breast' # Change to 'HTRU' or 'Credit' to produce plots for other datasets.
width = .35 # width of a bar

ax1=resS_pd[ds].plot(kind='bar', width = width, 
                                 figsize=(3,4),
                                 color = ('royalblue','skyblue'))
ax2=resA_pd[ds].plot(secondary_y=True, color = 'r', markersize=12, 
                             mew =2, marker = 'x', linewidth = 0)

plt.xlim([-width, 3-2*width])
ax2.set_ylim(0,1)
ax1.set_ylabel('Samples',fontsize = 14)
#ax1.set_ylim(0,100)

ax2.set_ylabel('Accuracy', color = 'r', fontsize = 14)
ax2.set_title(ds, fontsize = 14)
ax2.set_yticks(np.arange(0, 1.1, 0.1))

plt.grid(which='both',linestyle='--')
plt.show()